In [ ]:
if True:
    from julia.api import Julia
    jl = Julia(compiled_modules=False)

#import julia; julia.install(quiet=True)
from julia import Main

import numpy     as np
import panel     as pn; pn.extension()
import holoviews as hv; hv.extension( "bokeh", logo=False)

from panel.interact import interact

def raster(img):  return hv.Image(img).opts(cmap="gray", xaxis=None, yaxis=None, frame_width=200, aspect='equal')

In [ ]:
%load_ext julia.magic

In [4]:
%%julia
using Pkg
gen_la_pb_dir = "../GenLinAlgProblems"
Pkg.activate(gen_la_pb_dir)
# cd into GenLinAlgProblems, start julia, and add the package to dev:   pkg> dev .
using GenLinAlgProblems
using LinearAlgebra, LaTeXStrings, Latexify, NearestNeighbors;

  Activating project at `~/elementary-linear-algebra/GenLinAlgProblems`


<div style="height:2cm;">
<div style="float:center;width:100%;text-align:center;"><strong style="height:100px;color:darkred;font-size:40px;">The Conjugate Gradients Algorithm</strong>
</div></div>

> Find $x = argmin_x \Vert A x - b \Vert$ for positive definite $A$, i.e.,
minimize the objective function $f(x) = \frac{1}{2} x^t A x - b^t x$.

# A Orthogonality

Consider a **positive definite** matrix $A$ with Cholesky decomposition $A = R^t R$<br>
$\qquad$ and consider the inner product<br>
$\qquad < x, y > = x^t A y = ( R x )^t (R x)$

$\qquad$ Since $\; < x, x > = \Vert R x \Vert^2,\;\;$ this inner product can be used to define<br>
$\qquad$ the **$\mathbf{A}$-norm** $\Vert x \Vert_A = \sqrt{ x^t A x }$.

___
Define a new coordinate system $\tilde{x} = R x$.<br>
$\qquad$ (Remember $R$ is a square upper triangular matrix with positive values on the main diagonal, and hence invertible)

In this new coordinate system
* the level curves for $f(x) = x^t A x$ are circles:<br>
$\qquad x^t A x = x^t R^t R x = \tilde{x}^t \tilde{x}$
* Vectors $\tilde{x} = R x$ and $\tilde{y} = R y$ such that $y^t A x = 0$ are orthogonal:<br>
$\qquad y^t A x = 0 \Leftrightarrow \tilde{y}^t \tilde{x} = 0$<br>
$\qquad$ Such vectors are **$\mathbf{A}$-orthogonal** or **conjugate**.
* We can use the **Gram Schmidt** procedure to compute $A$-orthogonal vectors.

##### Display Code

In [5]:
# Plot Routines to show vector pairs in the original coordinate system and the x̃ = R x corrdinate system
def show_vecs(x_0,y_0, x_1, y_1):
    '''use hv.Path to show vectors from (x_0,y_0) to (x1, y1)'''
    return hv.Path( [ [[ x_0[i], y_0[i]],
                       [ x_1[i], y_1[i]]] for i in range(len(x_0))], ["x̃", "ỹ"] ).opts( aspect='equal')
def show_original_vecs( x_0, y_0, x_1, y_1, Rinv, length):
    '''use hv.Path to show A-orthogonal vectors, where A = R Rᵗ
       the arguments provide the vectors in the coordinate system defined by R:  x = R x̃
       to maintain a uniform vector length, lengths of the vectors in the original system are set to length
    '''
    def A_convert( x, y, Rinv):
        ''' convert a point back to the original coordinate system'''
        inv = Rinv @ np.vstack([x,y])
        return inv[0,:], inv[1,:]

    xa_0, ya_0 = A_convert( x_0, y_0, Rinv)
    xa_1, ya_1 = A_convert( x_1, y_1, Rinv)

    # adjust the (xa_1,xy_1) endpoint to be a distance length from (xa_0, ya_0)
    dx = xa_1 - xa_0
    dy = ya_1 - ya_0
    l  = length / np.hypot(dx, dy)
    return hv.Path([[[xa_0[i],            ya_0[i]],
                     [xa_0[i]+l[i]*dx[i], ya_0[i]+l[i]*dy[i] ]] for i in range(len(x_0)) ]).opts( aspect='equal')

In [6]:
R = np.array( [[1,0],[2,3]]); Rinv = np.linalg.inv(R)
A   = R @ R.T                                          # positive definite matrix A

# ===========================================================================================
N   = 16; l=0.2                          # number of vector pairs and vector length to display
θ   = np.linspace( 0, 2*np.pi, N+1)
x_0 = np.cos(θ); y_0 = np.sin(θ)         # place the vector pairs equispaced on a circlr in the x̃ = R x corrdinate system

# ===========================================================================================
# Show the plots
ha = \
show_original_vecs( x_0, y_0, (1+l)*x_0, (1+l)*y_0, Rinv, 0.2 )*\
show_original_vecs( x_0, y_0, x_0-l*y_0, y_0+l*x_0, Rinv, 0.2 )*\
hv.HLine(0)*hv.VLine(0)

horig= \
show_vecs( x_0, y_0, (1+l)*x_0, (1+l)*y_0 )*\
show_vecs( x_0, y_0, x_0-l*y_0, y_0+l*x_0 )*\
hv.HLine(0)*hv.VLine(0)

#### Example

This example works in reverse: given a set of orthogonal vectors along a circle in the $\tilde{x}, \tilde{y}$ coordinate system,<br>
$\qquad$ compute and display the corresponding **$\mathbf{A}$-orthogonal** vectors in the $x,y$ coordinate system.

In [7]:
h=\
ha.opts( title = "A-orthogonal Vectors")+horig.opts( title = "Converted to Orthogonal Vectors")
h.opts( hv.opts.HLine(line_width=0.2,color='black'), hv.opts.VLine(line_width=0.2,color='black'))

:Layout
   .Overlay.I  :Overlay
      .Path.I  :Path   [x,y]
      .Path.II :Path   [x,y]
      .HLine.I :HLine   [x,y]
      .VLine.I :VLine   [x,y]
   .Overlay.II :Overlay
      .Path.I  :Path   [x̃,ỹ]
      .Path.II :Path   [x̃,ỹ]
      .HLine.I :HLine   [x,y]
      .VLine.I :VLine   [x,y]

**Remark:** the vectors shown in blue line up with the origin in both coordinate systems.

# $\mathbf{A}$-orthogonal Search Directions

Consider the optimization problem $\; x = argmin_x \left( \frac{1}{2} x^t A x - b^t x\right)$,<br>
$\qquad$ where $A$ is positive definite. I.e., we are minimizing the objective function $f(x) = \frac{1}{2} x^t A x - b^t x$.

In **gradient descent,** an iteration step searches in the direction of the gradient of $f(x)$:<br>
$\qquad\ x_{k+1} = x_k - \mu_k \nabla f(x) = x_k + \mu_k (b - A x_k )$, i.e.,<br>
$\qquad$ the **search directions are the successive residual vectors** $r_k = A x_k - b$.

Let's try to generalize the iteration equation to $x_{k+1} = x_{k} + \mu_k\ p_k.\;$ where $p_k$ is the current search direction,<br>
$\qquad$ by finding the best point of the form $x_{k+1} = x_0 + \alpha_0 p_0 + \alpha_1 p_1 + \dots + \alpha_k p_k$,<br>
$\qquad$ where we have kept track of the previous search directions. Then

$\qquad f\left( x_{k+1} \right) = 
\frac{1}{2} x_0^t A x_0 - b^t x_0 + \sum_{i=0}^k{\left( \frac{1}{2} \alpha_i^2 p_i^t A p_i + \alpha_i p_i^t ( A x_0 - b)\right)}$<br><br>
$\qquad$ We see that if at the previous step $x_k \in x_0 + span\{ p_0, p_1, \dots, p_{k-1} \}$ was chosen to minimize $f(x_k)$,<br>
$\qquad$ then at the current step $\qquad\quad$ $x_{k+1} \in x_0 + span\{ p_0, p_1, \dots, p_{k-1}, p_k \}$ is chosen to minimize $f(x_{k+1})$.

$\qquad \therefore$ Progress in a new direction does not undo progress in the previous directions!

For the **Gradient Descent** algorithm, the successive search directions are the residuals $r_0, r_1, \dots, \text{ where } r_k = A x_{k}-b$.<br>
For the **Conjugate Gradients** algorithm, we will use the Gram-Schmidt procedure to make these successive search directions A-orthogonal. I.e.,<br>
$\qquad span\{ r_0, \dots r_k \} = span\{ p_0, \dots p_k \},$ where
$p_i^t A p_j = \left\{ \begin{align} \;& 1,\qquad & \text{ if } i = j \\
& 0 & \text{otherwise}\end{align}\right.$ 

<div style="background-color:#F2F5A9;color:black;">

**Theorem: (Conjugate Gradient Algorithm)** The residual $r_k = A x_k -b $ is in the Krylov subspace $K_{k+1} (A, r_0) = span\left\{ r_0, A r_0, A^2 r_0, \dots, A^{k} r_0 \right\}$

* $x_k = argmin_{x \in x_0+K_k(A,r_0)} f(x)$
* The residual $r_k$ is orthogonal to $K_k(A,r_0)$, and hence to $p_0, \dots p_{k-1}$ and $r_0, \dots r_{k-1}$.
* The residual $r_k$ is A-orthogonal to $K_{k-1}(A,r_0)$ and hence to $p_0,\dots, p_{k-2}$ and $r_0,\dots r_{k-2}$
* The search directions are A-orthogonal, i.e., for any $j < k$, $p_k$ is A-orthogonal to $p_j$.
</div>

Thus, if $r_k \ne 0$, the search directions $\{ p_0, p_1, \dots p_k \}$ form a basis for $K_{k+1} (A, r_0)$,<br>
$\qquad$ and for each $k,\; x_k - x_0 \in K_k (A, r_0)$.

For further information (and a proof of the above statements), see
* [H. Lee, Conjugate Gradient](https://services.math.duke.edu/~holee/math361-2020/lectures/Conjugate_gradients.pdf)
* [J. R. Shewchuk, Painless Conjugate Gradient](https://www.cs.cmu.edu/~quake-papers/painless-conjugate-gradient.pdf)

Pseudocode for the Conjugate Gradient algorithm
>
Given a symmetric positive definite matrix $A$, a vector $b$ and an initial value $x_0$<br>
Let $p_0 = r_0 = b - A x_0$<br>
for $k = 0, 1, \dots$<br>
    $\qquad$ let $\alpha_k = \frac{ r_k^t r_k}{p^t_k A p_k}$  $\qquad\qquad$ # exact line search<br>
    $\qquad$ let $x_{k+1} = x_k + \alpha_k p_k$<br>
    $\qquad$ let $r_{k+1} = b - A x_{k+1}$<br>
    $\qquad$ let $p_{k+1} = r_{k+1} + \frac{ r_k^t r_k }{ r^t_{k+1} r_{k+1}} p_k$  $\qquad$ # Gram-Schmidt<br>
end for

# Example

In [8]:
%%julia
function conjugate_gradients(A, b, x₀, max_iterations=100, tolerance=1e-6)
    f(x) = 0.5*x'*A*x-b'*x
    n = length(b)
    x = copy(x₀)
    r = b - A * x
    p = copy(r)
    residual_norm_squared = dot(r, r)
    tolerance_squared = tolerance * tolerance

    vals_x = [x]
    vals_y = [f(x)]
    
    for k in 1:max_iterations
        Ap = A * p
        alpha = residual_norm_squared / dot(p, Ap)
        x += alpha * p
        r -= alpha * Ap
        y=f(x); push!(vals_x,x); push!(vals_y,y)

        new_residual_norm_squared = dot(r, r)
        if new_residual_norm_squared < tolerance_squared
            return (x,y), (vals_x, vals_y)   # converged
        end

        beta = new_residual_norm_squared / residual_norm_squared
        p = r + beta * p
        residual_norm_squared = new_residual_norm_squared
    end
    
    return None, (vals_x, vals_y) # failed to converge
end;

In [10]:
%%julia
# Example usage
A = [4.0 -1.0; -1.0 3.0]
b = [1.0, 2.0]
x₀ = [0.0, 0.0]

println("Conjugate Gradient Solution: $(conjugate_gradients(A, b, x₀)[1][1])")
println("A\\b Solution:                $(A\b)")

Conjugate Gradient Solution: [0.4545454545454546, 0.8181818181818182]
A\b Solution:                [0.4545454545454546, 0.8181818181818182]


In [11]:
sol,vals=Main.conjugate_gradients( Main.A, Main.b, [3.,3.] )
vals[0]

[array([3., 3.]),
 array([0.33333333, 1.66666667]),
 array([0.45454545, 0.81818182])]

In [12]:
def obj_func( A, b, x ): return 0.5*(x.T @ (A @ x)) - b.T @ x

def show_objective_surface_contours(f,lims=(-3,3),n=101,l=[0.1,0.4,1,2,5,10,20]):

    from holoviews.operation import contours as hv_contours
    v   = np.linspace(*lims,n)
    x,y = np.meshgrid(v, v)
    z   = np.array(list(map( f, np.array([x.ravel(),y.ravel()]).T ))).reshape(x.shape)
    img = hv.Image((v,v,z), ["x_1", "x_2"], "y").opts(title="Contour Lines in x_1,x_2 plane")

    return img, hv_contours(img, levels=l, filled=True, overlaid=False).opts(padding=0)
    
def show_gradient_descent_on_contour(sol, steps):
    x_i = [p[0] for p in steps[0]]
    y_i = [p[1] for p in steps[0]]
    d_i = [np.linalg.norm(p) for p in steps[0]]
    progress = [ d_i[i]/d_i[i-1] for i in range(1,len(d_i)) ]
    
    if sol is not None: sol = np.round(sol[0], 5)

    _, hf = show_objective_surface_contours(f,lims=(-3,3), l=[0.1,0.4,1,2,5,10,20])
    return pn.Row( 
        hf.opts(width=350,height=350,xlim=(-3,3),ylim=(-3,3),title=f"Contour Lines, Solution {sol}" )\
        *hv.Curve(  (x_i, y_i)).opts(color="red")\
        *hv.Scatter((x_i, y_i)).opts(tools=['hover'],color="red", size=4),
        hv.Curve( steps[1], "index", "y" ).opts(height=350, title="y versus Iteration Index", tools=["hover"], show_grid=True),
        hv.Curve( (range(1,len(d_i)), progress), "index", "norm( x_i ) / norm(x_(i-1)"  ).opts(height=350, title="Relative Progress", tools=["hover"], show_grid=True)
    )

In [13]:
h=show_objective_surface_contours( lambda x: obj_func(Main.A, Main.b, x), l=[1,5,10,15]) #vals[1][::-1])[0]
h[1]

:Polygons   [x_1,x_2]   (y)

In [15]:
vals

([array([3., 3.]),
  array([0.33333333, 1.66666667]),
  array([0.45454545, 0.81818182])],
 array([13.5       ,  0.16666667, -1.04545455]))